In [ ]:
!pip install --no-cache-dir transformers sentencepiece &> /dev/null 
!pip install datasets &> /dev/null 
!pip install evaluate &> /dev/null 

# Supporting classes

## Spacehandler

In [ ]:
import re

"""
handles spaces before and after punctuation
functions:
- space_stripper - strips extra spaces from text, used in space_oddity
- space_oddity - adds extra spaces before punctuation for tokenization
- fried_nails - removes extra spaces before punctuation for anti-tokenization
"""
class SpaceHandler(object):
  def __init__(self):
    self.us = "[А-ЩЬЮЯЄҐІЇЭЫЪа-щьюяєґіїэыъ'0-9a-zA-Z()%‰\"№\+]" # ukrainian word symbol + brackets + quotation marks + percentage sign (ыыы костыль) + russian symbols (ik but it is what it is)
    self.upr = r'[.?!,;:—-]' # ukrainian punctuation
    self.uwr = re.compile(self.us + "+") # Matches a word. We want our model to predict hyphens, thus I remove - from here

  def space_stripper(self, sentence): # to get rid of extra spaces
      sentence = re.sub(r"\s{2,}", ' ', sentence) # double+ spaces
      sentence = re.sub(r"^\s+", '', sentence) # a space in the beginning (if double, then has already been removed)
      sentence = re.sub(r"\s+$", '', sentence) # a space in the end
      sentence = re.sub(r'([0-9])([.?!,;:—-])\s([0-9])', r"\1\2\3", sentence) # spaces in punctuation between numbers
      return sentence

  def space_oddity(self, sentence): # to add spaces in between of punctuation
      sentence = self.space_stripper(sentence) # get rid of extra spaces
      words = re.findall(self.uwr, sentence) # match words
      punctuation = re.split(self.uwr, sentence) # split the remains over words. The punctuation will be both at the beginning and in the end
      i = 0 # the index of considered punctuation
      sentence = "" # dummy for the newly created sentence
      while i < len(punctuation) - 1: # end before the last punctuation
        sentence += ' '.join(list(punctuation[i])) + ' ' +  words[i] + ' ' # the symbols between words now get to be joined by spaces. Likely with several spaces if there were spaces
        i += 1
      sentence += ' '.join(list(punctuation[-1])) # add the last punctuation to account for them not having the word following
      return self.space_stripper(sentence) # strip the remaining spaces just in case

  def fried_nails(self, sentence): # the reversed function: to remove the extra spaces. Not 1-to-1 (or onto?), like the previous function
      sentence = re.sub('\xad', '', sentence)
      words = re.findall(self.uwr, sentence) # retrieve the words as usual
      punctuation = re.split(self.uwr, sentence) # retrieve the rest
      i = 0
      sentence = ""
      while i < len(punctuation) -1:
        sentence += ''.join(re.split(r'\s+', punctuation[i])) + ' ' +  words[i] # now we remove the convenient spaces from punctuation, losing info
        i += 1
      sentence += ''.join(re.split(r'\s+', punctuation[-1]))
      sentence = re.sub(chr(8212), " " + chr(8212) + " ", sentence) # the dash must be separated at all times, no matter what
      sentence = re.sub(r'\s*-\s*', "-", sentence) # the hyphen is considered to cling always
      quote_split = re.split(r'\s*"\s*', sentence) # now, we deal with quotation marks
      sentence = ""
      for i in range(len(quote_split)//2):
          sentence += quote_split[2*i] + ' "' + quote_split[2*i+1] + '" ' # The odd numbered mark is the left one, the even numbered is the right one.
      if len(quote_split) % 2:
          sentence += quote_split[-1]
      # else: # if the number of marks is odd
          # print("Лапки порахуй, мудило") # A suggestion to the user: "Sorry, the program would work incorrectly if you do not fix the quotation marks yourself"
      sentence = re.sub(r"\s([.,;:?!])", r"\1", self.space_stripper(sentence)) # The rest of the punctuation gets clinged
      sentence = re.sub(r"\(\s+", '(', sentence) # fix the left brackets avoiding the "(" case (three punctuation marks in a row)
      sentence = re.sub(re.compile(f"({self.us})(\()"), r'\1 \2', sentence) # uncling the left bracket from a word
      sentence = re.sub("\s+\)", ')', sentence) # in the same way
      sentence = re.sub('–', ' –', sentence) # put space before tyre
      sentence = re.sub(re.compile(f"(\))({self.us})"), r'\1 \2', sentence) # uncling the right bracket from a word
      sentence = re.sub(r'\s*-\s*', "-", sentence) # the hyphen is considered to cling always
      sentence = re.sub(r"\s*’\s*", "’", sentence) # same for apostrophes
      sentence = re.sub(r'"\s\(', '"(', sentence) # removing the space from " (
      sentence = re.sub(r'\)\s"', ')"', sentence) # removing the space from ) "
      sentence = re.sub(r'([0-9])([.?!,;:—-])\s([0-9])', r"\1\2\3", sentence) # spaces in punctuation between numbers
      return sentence

##antichanger

In [ ]:
import time
import collections 
def antichanger(cor_word):    #тут буде генерація русизмів 
  start = time.time()
  res = set()
  res.add(cor_word.lower())

  one_replace_keys =["ё" , "мя" , "е" , "ё" , "ие" , "е", "ль" , "ы", "аи" , "ьо", "лья" , "ень" , "сч", "елье" , "нье", "e"]
  one_replace_values = ["е", "м'я", "є", "ьо", "іє", "і", "лль","и", "аї","йо", "л’я",  "інь", "щ", "ілля", "ння", "ьо"]

  
  def function_one_replace():
    for index, key in enumerate(one_replace_keys):
      temp = list()
      for st in res:
          x = st.replace(key,one_replace_values[index])
          temp.append(x)
      res.update(set(temp))

  def function_all_to_all():
    lt = "ауоиеіє"
    for c1 in lt:
      for c2 in lt:
        if c1 != c2:
          temp = list()
          for st in res:
              for i in range(len(st)) :
                if (st[i] == c1):
                    x = st[:i] + c2 + st[ i + 1:]
                    temp.append(x);
                x = st.replace(c1, c2)
                temp.append(x)
          res.update(set(temp))
          if len(res) > 10000:
            return 
  
  def function_all_ending():
      temp = list()
      dict_end = {"тись" : "ться", "овать" : "увати",  "ать" : "ати", "леть" : "літи",
                  "ити" : "ить", "ишь" : "іш","ый" : "ий", "ние" : "ння"}
      for st in res:
        for key in dict_end :
          pos = st.rfind(key)
          if pos != -1:
              x = st[:pos] + dict_end[key]
              temp.append(x)
      res.update(set(temp))
    
  def function_exeption():
    temp = list()
    for st in res:
        pos = st.find("бес")
        if pos != -1:
            x =  "без"+ st[pos+3:] 
            temp.append(x)

        pos = st.find("ис")
        if pos != -1 and pos + 2 < len(st):
            if (st[pos + 2] not in "ауеоиіїяює"):
              x =  "із"+ st[pos+2:] 
              temp.append(x)

        if(st[-1] == "я" and st[-2] in "ауеоиіїяює"):
          temp.append(st[:-1])
        if(st[-1] == "ц"):
          temp.append(st+"ь")
        

    res.update(set(temp))
  function_one_replace()
  
  function_all_ending()
  
  function_exeption()
  
  function_all_to_all()
  
  return set(res)

## Errorifier rus

In [ ]:
# impport list of all ukr words
words = list()
with open('/content/drive/MyDrive/ukramarly-main/spelling/wordlist.txt') as f:
    for line in f:
      if line:
        words.append(line.replace("\n", ""))
words = set(words)

#import frequency vocab
myfile = open("/content/drive/MyDrive/artem-yushko/final-everything/datasets/frequency-vocab.txt", 'r')
frequence_dict = {}
for line in myfile:
    k, v = line.strip().split()
    frequence_dict[k.strip()] = int(v.strip())

vocab = set(frequence_dict.keys())

In [ ]:
"For getting prob of error"
import random
def random_replace(prob):
  gen = random.randint(0, 100) / 100
  return gen < prob

"For batch translating ukr to rus"
from transformers import pipeline
translator = pipeline(task="translation", model='Helsinki-NLP/opus-mt-uk-ru', device=0)

def translate_and_append(words_to_errorify):
  wordset = [word[0] for word in words_to_errorify]
  translated = translator(wordset, batch_size=64)
  translated = [sent['translation_text'].lower() for sent in translated]
  for i in range(len(words_to_errorify)):
    words_to_errorify[i].append(translated[i])
  return words_to_errorify

space_handler = SpaceHandler()

/usr/local/lib/python3.8/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
'Main function. Takes in list of corr sentences, outputs list of incorr sentences'
def errorify_rus_dataset(dataset, prob):
  all_sentences = [] #list of all tokenized sentences
  for sentence in dataset:
    all_sentences.append(sentence.split(' '))

  # list of all words to errorify - we later use batch translation on them
  words_to_errorify = []
  for sent_id in range(len(all_sentences)):    
    for word_id in range(len(all_sentences[sent_id])):
        if random_replace(prob):
          words_to_errorify.append([all_sentences[sent_id][word_id], sent_id, word_id]) #first item on the list is ukr_word, second is sent_id, third is word_id

  # batch translate all words and append translated as final one
  words_to_errorify = translate_and_append(words_to_errorify)

  # for each word in words to errorify, errorify and replace it in the original dataset
  for word_lst in words_to_errorify:
    word_ukr = word_lst[0]
    work_rus = word_lst[3]
    sent_id = word_lst[1]
    word_id = word_lst[2]
    is_capital = (word_ukr.capitalize() == word_ukr)
    if word_ukr == work_rus or len(word_ukr) < 2:
      continue
    interim = (set(antichanger(work_rus) )& vocab ) - words - set([work_rus]) 
    new_dict = {i:frequence_dict[i] for i in (interim)}
    variants = [pair[0] for pair in sorted(new_dict.items(), key=lambda item: -item[1])]
    i = random.randint(0, (len(variants)-1) %4)
    # if variants[i] does not exist (i dont know why it wouldn't) we skip
    if len(variants)==0:
      continue
    suggested_surzhik = variants[i]
    if is_capital:
      suggested_surzhik = suggested_surzhik.capitalize()
    all_sentences[sent_id][word_id] = suggested_surzhik

  output_sentences = []
  for sentence in all_sentences:
    output_sentences.append(space_handler.fried_nails(" ".join(sentence)).replace("ʼ ", "ʼ"))
  return output_sentences

# Errorify dataset

In [ ]:
# clean data FILE to be errorified
input_file = "/content/drive/MyDrive/artem-yushko/data-artem/cleaned/borshch4.txt"

# output FOLDER for the errorified and tagged data, future model input
out_folder = "/content/drive/MyDrive/datasets/preprocessed/synth-1"

In [ ]:
import os
# creating the output folder
if not os.path.exists(out_folder):
  os.mkdir(out_folder)

# reading the file
with open(input_file, 'r') as f:
  text = f.read()
  lines = text.split('\n')

In [ ]:
dataset = lines[:10000]

In [ ]:
import time
s = time.time()
out = errorify_rus_dataset(dataset,0.01)
print(time.time() - s)

OutOfMemoryError: ignored

In [ ]:
output_sentences = [sent + '\n' for sent in out]
with open(out_folder + "/russism-applied", 'w') as f:
  f.writelines(output_sentences)